# Imports

In [1]:
import numpy as np
from functools import partial

In [2]:
from copy import deepcopy
from functools import partial
from pprint import pprint
import os.path as osp

In [3]:
import sacred
import torch
import torch.nn as nn
import torch.nn.functional as F
from tqdm import tqdm

from torch.backends import cudnn

In [4]:
from torch.utils.data import DataLoader, RandomSampler, BatchSampler
from typing import NamedTuple, Optional, List

In [5]:
from models.matcher import MatchERT
from models.ingredient import model_ingredient, get_model

In [6]:
from sacred import SETTINGS
from sacred.utils import apply_backspaces_and_linefeeds
from sacred import Ingredient

In [7]:
from torch.optim import Optimizer
from torch.optim.lr_scheduler import _LRScheduler
from torch.utils.data import DataLoader, Dataset
from utils.metrics import *

/mnt/beegfs/home/smessoud/anaconda3/envs/rrt/lib/python3.7/site-packages/ranx/qrels_run_common.py:7: UserWarning: Sorting disabled. Assumes that you provided sorted doc_ids!
  warnings.warn("Sorting disabled. Assumes that you provided sorted doc_ids!")


In [8]:
from utils import pickle_load
from sacred import Experiment
from utils.data.dataset_ingredient import data_ingredient, get_loaders
from utils.data.dataset import FeatureDataset
ex = sacred.Experiment('RRT Evaluation', ingredients=[data_ingredient, model_ingredient], interactive=True)

In [9]:
ex = sacred.Experiment('Prepare Top-K (VIQUAE FOR RTT)', interactive=True)
# Filter backspaces and linefeeds
SETTINGS.CAPTURE_MODE = 'sys'
ex.captured_out_filter = apply_backspaces_and_linefeeds

In [10]:
feature_name = 'r50_gldv1'
set_name = 'train'
gnd_name = 'gnd_'+ set_name+'.pkl'

In [11]:
cpu = False
cudnn_flag = 'benchmark'
temp_dir = osp.join('logs', 'temp')
resume = '/mnt/beegfs/home/smessoud/RerankingTransformer/RRT_GLD/rrt_gld_ckpts/r50_gldv1.pt'
seed = 0

In [33]:
dataset_name = 'viquae_for_rrt'
data_dir = osp.join('/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data', dataset_name)

In [34]:
train_data_dir = '/mnt/beegfs/home/smessoud/RerankingTransformer/models/research/delf/delf/python/delg/data/viquae_for_rrt'

In [35]:
gnd = pickle_load(osp.join(train_data_dir, gnd_name))

In [36]:
len(gnd['imlist'])

42678

In [61]:
gnd['gnd'][47]

{'easy': ['512px-Achille_Compagnoni_1955.jpg',
  '512px-K2_2006b.jpg',
  '512px-Lino_Lacedelli_1954.jpg'],
 'hard': [],
 'junk': ['512px-Everest_kalapatthar_crop.jpg',
  '512px-Panoramic_view_of_Mussoorie,_Uttarakhand.jpg',
  '512px-Dolakha_Jiri.jpg',
  '512px-Cho_Oyu.jpg',
  '512px-210th_Rescue_Squadron_-_1.jpg',
  '512px-The_Malaysian_Federal_Honour_-_Panglima_Jasa_Negara_(PJN).jpg',
  '512px-Hugh_Ruttledge_1936.jpg',
  '512px-Lhotse.jpg',
  '512px-Douglas_Mawson_resting.jpg',
  '512px-Annapurna_I.jpg',
  '512px-Chris_bundled_up_on_an_expedition.jpg',
  '512px-Andrew_Scott_Waugh_1861.jpg',
  '512px-Padma_Bhushan_Bachendri_Pal_(cropped).jpg',
  '512px-15-152-Pluto-NewHorizons-HighResolution-20150714-IFV.jpg',
  '512px-G3C_spacesuit_Gemini_3.jpg',
  '512px-Mount_Everest_as_seen_from_Drukair2_PLW_edit.jpg',
  '512px-George_Everest_-_Maull_&_Polyblank.jpg',
  "512px-Joske's_Thumb_from_near_Montfort_Boys_Town,_Fiji.jpg",
  '512px-Kobe_eddiebauer2560.jpg',
  '512px-Carstensz-expeditie.jpg'

In [52]:
i = 47
for j in gnd['gnd'][i]['r_easy']:
    new_query = gnd['gnd'][i]['rank_img_dict'][j]
    print(new_query)

512px-Achille_Compagnoni_1955.jpg
512px-K2_2006b.jpg
512px-Lino_Lacedelli_1954.jpg


In [59]:
i = 47
for j in gnd['gnd'][i]['r_easy']:
    new_query = gnd['gnd'][i]['rank_img_dict'][j]
    index = gnd['gnd'][i]['img_rank_dict'][new_query]
    print(new_query)
    print(index)
    gnd['simlist'][i][index]


512px-Achille_Compagnoni_1955.jpg
22


IndexError: list index out of range

In [65]:
gnd['gnd'][i]['r_easy'], gnd['gnd'][i]['r_junk'], 

([22, 24, 5],
 [23,
  19,
  3,
  21,
  12,
  10,
  20,
  11,
  18,
  2,
  8,
  9,
  15,
  17,
  4,
  6,
  7,
  0,
  1,
  16,
  14,
  13])

In [64]:
not('512px-Achille_Compagnoni_1955.jpg' in gnd['simlist'][i])

True

In [57]:
len(gnd['simlist'][i])

22

In [54]:
[12, 8, 4, 15, 11, 23, 17]

[12, 8, 4, 15, 11, 23, 17]